In [1]:
import pandas as pd
import os

In [131]:
pd.set_option('max_rows', 20)

In [3]:
%load_ext dotenv
%dotenv

%load_ext autoreload
%autoreload 2

In [4]:
from sshtunnel import SSHTunnelForwarder

AWS_IP_ADDRESS = '54.185.35.146'
AWS_USERNAME = os.environ.get('AWS_USERNAME')
SSH_KEY_PATH = '/Users/brenner/.ssh/id_rsa'

server = SSHTunnelForwarder(
    AWS_IP_ADDRESS,
    ssh_username=AWS_USERNAME,
    ssh_pkey=SSH_KEY_PATH,
    remote_bind_address=('localhost', 5432),
)

server.start()
print(server.is_active, server.is_alive, server.local_bind_port)

True True 65173


In [13]:
from sqlalchemy import create_engine

# Postgres username, password, and database name
POSTGRES_IP_ADDRESS = 'localhost' ## This is localhost because SSH tunnel is active
POSTGRES_PORT = str(server.local_bind_port)
POSTGRES_USERNAME = os.environ.get('POSTGRES_USERNAME')     ## CHANGE THIS TO YOUR POSTGRES USERNAME
POSTGRES_PASSWORD = os.environ.get('POSTGRES_PASSWORD') ## CHANGE THIS TO YOUR POSTGRES PASSWORD
POSTGRES_DBNAME = 'tennis'

# A long string that contains the necessary Postgres login information
postgres_str = ('postgresql://{username}:{password}@{ipaddress}:{port}/{dbname}'
                .format(username=POSTGRES_USERNAME, 
                        password=POSTGRES_PASSWORD,
                        ipaddress=POSTGRES_IP_ADDRESS,
                        port=POSTGRES_PORT,
                        dbname=POSTGRES_DBNAME))

# Create the connection
cnx = create_engine(postgres_str)

## Load Some Data!

Pandas has a `read_sql_query` method that will pass a SQL statement to a database connection. Here is an example from the all-star table.

In [16]:
pd.read_sql_query('''
SELECT *
FROM aus_women_2013
''', cnx)

,player1,player2,round,result,fnl1,fnl2,fsp_1,fsw_1,ssp_1,ssw_1,...,bpc_2,bpw_2,npa_2,npw_2,tpw_2,st1_2,st2_2,st3_2,st4_2,st5_2
0,Serena Williams,Ashleigh Barty,1,1,2.0,0.0,59.0,20.0,41.0,8.0,...,0.0,0.0,2.0,4.0,31,2.0,1.0,NaN,None,None
1,Vesna Dolonc,Lara Arruabarrena,1,1,2.0,1.0,65.0,33.0,35.0,10.0,...,4.0,7.0,NaN,NaN,74,6.0,2.0,4.0,None,None
2,Pauline Parmentier,Karolina Pliskova,1,0,0.0,2.0,63.0,16.0,37.0,4.0,...,5.0,14.0,NaN,NaN,64,6.0,6.0,NaN,None,None
3,Heather Watson,Daniela Hantuchova,1,0,1.0,2.0,61.0,41.0,39.0,19.0,...,5.0,13.0,5.0,8.0,102,7.0,3.0,6.0,None,None
4,Samantha Stosur,Klara Zakopalova,1,1,2.0,0.0,65.0,28.0,35.0,11.0,...,4.0,14.0,10.0,15.0,60,3.0,4.0,NaN,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
122,Simona Halep,Dominika Cibulkova,5,0,0.0,2.0,67.0,13.0,33.0,6.0,...,5.0,9.0,3.0,4.0,54,6.0,6.0,NaN,None,None
123,Agnieszka Radwanska,Victoria Azarenka,5,1,2.0,1.0,59.0,33.0,41.0,16.0,...,2.0,5.0,20.0,34.0,74,1.0,7.0,0.0,None,None
124,Eugenie Bouchard,Na Li,6,0,0.0,2.0,45.0,13.0,55.0,5.0,...,6.0,10.0,11.0,14.0,71,6.0,6.0,NaN,None,None
125,Dominika Cibulkova,Agnieszka Radwanska,6,1,2.0,0.0,64.0,22.0,36.0,10.0,...,1.0,9.0,4.0,9.0,40,1.0,2.0,NaN,None,None


In [17]:
'''CREATE TABLE  aus_men_2013 (
      player1 VARCHAR(255)DEFAULT NULL,
      player2 VARCHAR(255)DEFAULT NULL,
      round VARCHAR DEFAULT NULL,
      result VARCHAR DEFAULT NULL,
      fnl1 DOUBLE PRECISION DEFAULT NULL,
      fnl2 DOUBLE PRECISION DEFAULT NULL,
      fsp_1 DOUBLE PRECISION DEFAULT NULL,
      fsw_1 DOUBLE PRECISION DEFAULT NULL,
      ssp_1 DOUBLE PRECISION DEFAULT NULL,
      ssw_1 DOUBLE PRECISION DEFAULT NULL,
      ace_1 VARCHAR DEFAULT NULL,
      dbf_1 VARCHAR DEFAULT NULL,
      wnr_1 VARCHAR DEFAULT NULL,
      ufe_1 VARCHAR DEFAULT NULL,
      bpc_1 VARCHAR DEFAULT NULL,
      bpw_1 VARCHAR DEFAULT NULL,
      npa_1 VARCHAR DEFAULT NULL,
      npw_1 VARCHAR DEFAULT NULL,
      tpw_1 VARCHAR DEFAULT NULL,
      st1_1 VARCHAR DEFAULT NULL,
      st2_1 VARCHAR DEFAULT NULL,
      st3_1 VARCHAR DEFAULT NULL,
      st4_1 VARCHAR DEFAULT NULL,
      st5_1 VARCHAR DEFAULT NULL,
      fsp_2 DOUBLE PRECISION DEFAULT NULL,
      fsw_2 DOUBLE PRECISION DEFAULT NULL,
      ssp_2 DOUBLE PRECISION DEFAULT NULL,
      ssw_2 DOUBLE PRECISION DEFAULT NULL,
      ace_2 VARCHAR DEFAULT NULL,
      dbf_2 VARCHAR DEFAULT NULL,
      wnr_2 VARCHAR DEFAULT NULL,
      ufe_2 VARCHAR DEFAULT NULL,
      bpc_2 VARCHAR DEFAULT NULL,
      bpw_2 VARCHAR DEFAULT NULL,
      npa_2 VARCHAR DEFAULT NULL,
      npw_2 VARCHAR DEFAULT NULL,
      tpw_2 VARCHAR DEFAULT NULL,
      st1_2 VARCHAR DEFAULT NULL,
      st2_2 VARCHAR DEFAULT NULL,
      st3_2 VARCHAR DEFAULT NULL,
      st4_2 VARCHAR DEFAULT NULL,
      st5_2 VARCHAR DEFAULT NULL);'''

'CREATE TABLE  aus_men_2013 (\n      player1 VARCHAR(255)DEFAULT NULL,\n      player2 VARCHAR(255)DEFAULT NULL,\n      round VARCHAR DEFAULT NULL,\n      result VARCHAR DEFAULT NULL,\n      fnl1 DOUBLE PRECISION DEFAULT NULL,\n      fnl2 DOUBLE PRECISION DEFAULT NULL,\n      fsp_1 DOUBLE PRECISION DEFAULT NULL,\n      fsw_1 DOUBLE PRECISION DEFAULT NULL,\n      ssp_1 DOUBLE PRECISION DEFAULT NULL,\n      ssw_1 DOUBLE PRECISION DEFAULT NULL,\n      ace_1 VARCHAR DEFAULT NULL,\n      dbf_1 VARCHAR DEFAULT NULL,\n      wnr_1 VARCHAR DEFAULT NULL,\n      ufe_1 VARCHAR DEFAULT NULL,\n      bpc_1 VARCHAR DEFAULT NULL,\n      bpw_1 VARCHAR DEFAULT NULL,\n      npa_1 VARCHAR DEFAULT NULL,\n      npw_1 VARCHAR DEFAULT NULL,\n      tpw_1 VARCHAR DEFAULT NULL,\n      st1_1 VARCHAR DEFAULT NULL,\n      st2_1 VARCHAR DEFAULT NULL,\n      st3_1 VARCHAR DEFAULT NULL,\n      st4_1 VARCHAR DEFAULT NULL,\n      st5_1 VARCHAR DEFAULT NULL,\n      fsp_2 DOUBLE PRECISION DEFAULT NULL,\n      fsw_2 DOUBLE P

In [24]:
query = '''
SELECT 
      player1, player2, result 
FROM 
      us_men_2013 
LIMIT 5;
'''

pd.read_sql_query(query, cnx)

,player1,player2,result
0,Richard Gasquet,Michael Russell,1
1,Stephane Robert,Albano Olivetti,1
2,Jan-Lennard Struff,Guillaume Rufin,0
3,Aljaz Bedene,Dmitry Tursunov,0
4,Feliciano Lopez,Florent Serra,1


In [32]:
query = '''
SELECT player1, player2, result
FROM us_men_2013 
WHERE player1 = 'Richard Gasquet';
'''

pd.read_sql_query(query, cnx)

,player1,player2,result
0,Richard Gasquet,Michael Russell,1
1,Richard Gasquet,Stephane Robert,1
2,Richard Gasquet,Dmitry Tursunov,1
3,Richard Gasquet,Milos Raonic,1
4,Richard Gasquet,David Ferrer,1
5,Richard Gasquet,Rafael Nadal,0


In [33]:
query = '''
SELECT COUNT(*)
FROM us_men_2013;
'''

pd.read_sql_query(query, cnx)

,count
0,126


In [35]:
query = '''
SELECT 
      player1, player2, result 
FROM 
      us_men_2013 
WHERE 
      result = 1 
LIMIT 5;
'''

pd.read_sql_query(query, cnx)

,player1,player2,result
0,Richard Gasquet,Michael Russell,1
1,Stephane Robert,Albano Olivetti,1
2,Feliciano Lopez,Florent Serra,1
3,Mikhail Kukushkin,Andrej Martin,1
4,Roberto Bautista Agut,Thomaz Bellucci,1


In [37]:
query = '''
SELECT player1, AVG(result) 
FROM us_men_2013 
GROUP BY player1;
'''

pd.read_sql_query(query, cnx)

,player1,avg
0,Jiri Vesely,0.000000
1,Tommy Haas,0.666667
2,Sergiy Stakhovsky,0.000000
3,Paolo Lorenzi,0.000000
4,Jurgen Zopp,0.000000
...,...,...
76,Victor Hanescu,0.000000
77,Guillaume Rufin,0.000000
78,Ryan Harrison,0.000000
79,Andreas Seppi,0.666667


In [38]:
query = '''
SELECT 
      us_men_2013.player1, us_men_2013.tpw_1, french_men_2013.tpw_1 
FROM 
      us_men_2013, french_men_2013 
WHERE 
      us_men_2013.player1 = french_men_2013.player1;
'''

pd.read_sql_query(query, cnx)

,player1,tpw_1,tpw_1
0,Tobias Kamke,62,140
1,Tobias Kamke,157,140
2,Julien Benneteau,63,163
3,Julien Benneteau,98,163
4,Julien Benneteau,125,163
...,...,...,...
139,Novak Djokovic,165,158
140,Novak Djokovic,115,158
141,Novak Djokovic,94,158
142,Novak Djokovic,102,158


In [180]:
query = '''
SELECT player, fsp_1
FROM
(SELECT fsp_1, player1 as player
FROM us_men_2013
UNION ALL
SELECT fsp_1, player2 as player
FROM us_men_2013
UNION ALL
SELECT fsp_1, player1 as player
FROM french_men_2013
UNION ALL
select fsp_1, player2 as player
FROM french_men_2013
UNION ALL
SELECT fsp_1, player1 as player
FROM aus_men_2013
UNION ALL
select fsp_1, player2 as player
FROM aus_men_2013
UNION ALL
SELECT fsp_1, player1 as player
FROM wimbledon_men_2013
UNION ALL
select fsp_1, player2 as player
FROM wimbledon_men_2013) AS t_1
ORDER BY fsp_1 DESC
'''

pd.read_sql_query(query, cnx)

,player,fsp_1
0,R.Federer,85.0
1,V.Hanescu,85.0
2,Daniel Brands,84.0
3,Rafael Nadal,84.0
4,Roberto Bautista Agut,83.0
5,Gilles Muller,83.0
6,Julien Benneteau,82.0
7,Roger Federer,82.0
8,Tim Smyczek,82.0
9,Roberto Bautista Agut,82.0


In [149]:
query = '''
SELECT * FROM
(SELECT player1, COUNT(round)
FROM us_men_2013
GROUP BY player1) as t_1
FULL OUTER JOIN
(SELECT player2, COUNT(round)
FROM us_men_2013
GROUP BY player2) as t_2
ON t_1.player1 = t_2.player2
'''

pd.read_sql_query(query, cnx)

ProgrammingError: (psycopg2.ProgrammingError) syntax error at or near "as"
LINE 9: GROUP BY player2) as t_2
                          ^
 [SQL: '\nSELECT * FROM\n(SELECT player1, COUNT(round)\nFROM us_men_2013\nGROUP BY player1) as t_1\nUNION ALL\n(SELECT player2, COUNT(round)\nFROM us_men_2013\nGROUP BY player2) as t_2\nON t_1.player1 = t_2.player2\n'] (Background on this error at: http://sqlalche.me/e/f405)

In [106]:
query = '''
CREATE VIEW view1
SELECT player1, COUNT(tpw_1)
FROM us_men_2013
GROUP BY player1
'''

pd.read_sql_query(query, cnx)

ProgrammingError: (psycopg2.ProgrammingError) syntax error at or near "SELECT"
LINE 3: SELECT player1, COUNT(tpw_1)
        ^
 [SQL: '\nCREATE VIEW view1\nSELECT player1, COUNT(tpw_1)\nFROM us_men_2013\nGROUP BY player1\n'] (Background on this error at: http://sqlalche.me/e/f405)

In [158]:
query = '''
SELECT *
FROM us_men_2013
'''

pd.read_sql_query(query, cnx)

,player1,player2,round,result,fnl1,fnl2,fsp_1,fsw_1,ssp_1,ssw_1,...,bpc_2,bpw_2,npa_2,npw_2,tpw_2,st1_2,st2_2,st3_2,st4_2,st5_2
0,Richard Gasquet,Michael Russell,1,1,3.0,0.0,63.0,45.0,37.0,16.0,...,1,3,30.0,40.0,83,3,4,2.0,NaN,NaN
1,Stephane Robert,Albano Olivetti,1,1,3.0,0.0,61.0,44.0,39.0,19.0,...,0,1,NaN,NaN,71,3,3,4.0,NaN,NaN
2,Jan-Lennard Struff,Guillaume Rufin,1,0,2.0,3.0,55.0,61.0,45.0,32.0,...,5,15,NaN,NaN,149,7,6,2.0,2.0,6.0
3,Aljaz Bedene,Dmitry Tursunov,1,0,1.0,3.0,52.0,41.0,48.0,19.0,...,6,9,NaN,NaN,121,7,4,6.0,6.0,NaN
4,Feliciano Lopez,Florent Serra,1,1,3.0,1.0,58.0,54.0,42.0,30.0,...,0,3,NaN,NaN,123,7,2,3.0,3.0,NaN
5,Kenny De Schepper,Bradley Klahn,1,0,1.0,3.0,59.0,68.0,41.0,37.0,...,3,11,30.0,48.0,151,6,6,7.0,7.0,NaN
6,Andrey Kuznetsov,Dudi Sela,1,0,2.0,3.0,53.0,59.0,47.0,44.0,...,11,26,NaN,NaN,187,7,6,6.0,5.0,6.0
7,Pablo Cuevas,Janko Tipsarevic,1,0,1.0,2.0,51.0,39.0,49.0,24.0,...,2,9,10.0,10.0,104,6,6,6.0,NaN,NaN
8,Ernests Gulbis,Andreas Haider-Maurer,1,0,2.0,3.0,58.0,68.0,42.0,31.0,...,4,14,NaN,NaN,160,3,6,1.0,7.0,6.0
9,Mikhail Kukushkin,Andrej Martin,1,1,3.0,0.0,51.0,35.0,49.0,27.0,...,7,12,NaN,NaN,103,4,6,5.0,NaN,NaN


In [57]:
query = '''
SELECT
      us_men_2013.player1, 
      SUM(us_men_2013.tpw_1) AS us_points, 
      SUM(french_men_2013.tpw_1) AS french_points, 
      SUM(us_men_2013.tpw_1 + french_men_2013.tpw_1) AS total_points 
FROM
      us_men_2013, french_men_2013 
WHERE 
      us_men_2013.player1 = french_men_2013.player1
GROUP BY
      us_men_2013.player1;
'''

pd.read_sql_query(query, cnx)

,player1,us_points,french_points,total_points
0,Jiri Vesely,112,120,232
1,Tommy Haas,1208,1569,2777
2,Sergiy Stakhovsky,156,57,213
3,Jurgen Zopp,116,58,174
4,Ernests Gulbis,159,97,256
...,...,...,...,...
37,Tommy Robredo,846,860,1706
38,Albert Montanes,57,63,120
39,Victor Hanescu,156,71,227
40,Andreas Seppi,1185,1263,2448


In [75]:
query = '''
SELECT 
      us_men_2013.player1, 
      SUM(us_men_2013.tpw_1) AS us_points, 
      SUM(french_men_2013.tpw_1) AS french_points
FROM 
      us_men_2013, french_men_2013 
WHERE 
      us_men_2013.player1 = french_men_2013.player1 
GROUP BY 
      us_men_2013.player1, us_men_2013.player2;
'''

pd.read_sql_query(query, cnx)

,player1,us_points,french_points
0,Adrian Mannarino,157,160
1,Ivan Dodig,93,172
2,Stanislas Wawrinka,444,584
3,Tommy Robredo,300,215
4,Victor Hanescu,156,71
...,...,...,...
61,Jurgen Zopp,116,58
62,Feliciano Lopez,282,206
63,Evgeny Donskoy,111,120
64,Ernests Gulbis,159,97
